## Prerequisites

1. Go to https://colab.research.google.com/
2. Upload this notebook with `File` -> `Upload Notebook` and upload this notebook

In [ ]:
!uv pip install -q "git+https://github.com/OpenMined/syft-flwr.git@main"

## Login to Datasite

In [ ]:
import syft_client as sc

import syft_flwr

print(f"{sc.__version__ = }")
print(f"{syft_flwr.__version__ = }")

do_email = input("Enter the Data Owner's email: ")
do_client = sc.login_do(email=do_email)

## Peers

First, the DS needs to add DO1 add a peer (see DS notebook)

In [ ]:
# should print out the DS peer
do_client.peers

In [ ]:
do_client.approve_peer_request(do_client.peers[0].email)

## Dataset

**Note: Please added this DO as peer on the DS side before uploading the dataset. If not, the DS will not be able to discover the dataset (it's a bug for now)**

In [ ]:
from pathlib import Path

from huggingface_hub import snapshot_download

# download the dataset from huggingface
DATASET_DIR = Path("./dataset/").expanduser().absolute()

if not DATASET_DIR.exists():
    snapshot_download(
        repo_id="khoaguin/pima-indians-diabetes-database-partitions",
        repo_type="dataset",
        local_dir=DATASET_DIR,
    )


# create a syft dataset (with mock and private path)
partition_number = 0
DATASET_PATH = DATASET_DIR / f"pima-indians-diabetes-database-{partition_number}"

do_client.create_dataset(
    name="pima-indians-diabetes-database",
    mock_path=DATASET_PATH / "mock",
    private_path=DATASET_PATH / "private",
    summary="This is a partition of the pima-indians-diabetes-database",
    readme_path=DATASET_PATH / "README.md",
    tags=["diabetes, health, medicine"],
    sync=True,
)

do_client.datasets.get_all()

## Jobs

Check for submitted jobs

In [ ]:
do_client.jobs

Approve submitted jobs

In [ ]:
do_client.jobs[0].approve()

In [ ]:
do_client.jobs

Run submitted jobs

In [ ]:
do_client.process_approved_jobs()

Check jobs status

In [ ]:
do_client.jobs

## Clean up

In [ ]:
do_client.delete_syftbox()